# Group by Operations

After loading, merging, and preparing a data set, a familiar task is to compute group statistics or possibly pivot tables for reporting or visualization purposes. pandas provides a flex- ible and high-performance groupby facility, enabling you to slice and dice, and sum- marize data sets in a natural way.

After loading, merging, and preparing a data set, a familiar task is to compute group statistics or possibly pivot tables for reporting or visualization purposes. pandas provides a flex- ible and high-performance groupby facility, enabling you to slice and dice, and sum- marize data sets in a natural way.

Hadley Wickham, an author of many popular packages for the R programming lan- guage, coined the term split-apply-combine for talking about group operations, and I think that’s a good description of the process. In the first stage of the process, data contained in a pandas object, whether a Series, DataFrame, or otherwise, is split into groups based on one or more keys that you provide. The splitting is performed on a particular axis of an object. For example, a DataFrame can be grouped on its rows (axis=0) or its columns (axis=1). Once this is done, a function is applied to each group, producing a new value. Finally, the results of all those function applications are com- bined into a result object

## Split

In [3]:
from pandas import Series, DataFrame
import pandas as pd
import numpy as np
df = DataFrame({'key1' : ['a', 'a', 'b', 'b', 'a'],
                'key2' : ['one', 'two', 'one', 'two', 'one'],
                'data1' : np.random.randn(5), 'data2' : np.random.randn(5)})
df

,data1,data2,key1,key2
0,1.304037,0.769563,a,one
1,0.170739,1.491167,a,two
2,1.105145,-1.906942,b,one
3,-1.027913,-0.610946,b,two
4,0.302438,0.528279,a,one


The split works with any of the axis. Simply put, you must give the groupby operator something to compute an array that identifies the group for each of the components of the axis belongs. For example, you can group the observations (axis = 0) by the values they take in one or more columns:

In [5]:
grouped = df.groupby(['key1', 'key2'])

In [8]:
for (x1, x2), group in grouped:
    print(x1, x2)
    print(group)

a one
      data1     data2 key1 key2
0  1.304037  0.769563    a  one
4  0.302438  0.528279    a  one
a two
      data1     data2 key1 key2
1  0.170739  1.491167    a  two
b one
      data1     data2 key1 key2
2  1.105145 -1.906942    b  one
b two
      data1     data2 key1 key2
3 -1.027913 -0.610946    b  two


Of course, you can choose to do whatever you want with the pieces of data. A recipe you may find useful is computing a dict of the data pieces as a one-liner:

In [10]:
pieces = dict(list(df.groupby(['key1', 'key2'])))
pieces[('b', 'one')]    

,data1,data2,key1,key2
2,1.105145,-1.906942,b,one


We can also group by the levels of a hierarchical index:

In [24]:
import pandas_datareader.data as web
end = '2015-01-01'
start = '2007-01-01'
get_px = lambda x: web.DataReader(x, 'yahoo', start=start, end=end)['Adj Close']
symbols = ['SPY','TLT','MSFT']
# raw adjusted close prices
data = pd.DataFrame({sym:get_px(sym) for sym in symbols})
data = data.reset_index()
data2 = pd.melt(data, id_vars ='Date',var_name = 'Index', value_name = 'Value')
data3 = data2.set_index(['Index', 'Date'])
data3.head()

Value
Index Date                 
MSFT  2007-01-03  23.478417
      2007-01-04  23.439102
      2007-01-05  23.305433
      2007-01-08  23.533456
      2007-01-09  23.557044

In [13]:
for name, group in data3.groupby(level= 'Index'):
    print(name)

MSFT
SPY
TLT


We can also group by applying a function to the index and use the resulting array. For example:

In [26]:
data5 = data.set_index('Date')
data5.groupby(lambda x: x.year).mean()

,MSFT,SPY,TLT
2007,24.110418,120.869750,64.303781
2008,21.393863,101.762682,72.870493
2009,18.909635,81.340931,77.390758
2010,22.692035,99.744057,79.427345
2011,22.371939,113.035932,88.143651
2012,26.308262,125.637040,107.927967
2013,29.557505,152.926545,101.144856
2014,39.738981,183.191441,106.679183


### Syntactic sugar

Most of the time, it's not necessary to take all the columns with you. 

df.groupby('key1')['data1'] == df['data1'].groupby(df['key1'])

## Apply

Once you have the data split by groups, the next step is to apply a function to each group.

### Aggregate

Aggregate functions are the type of functions that take an array and return a scalar. Thus, for each group, these functions will return a scalar. 

In [16]:
data3.groupby(level = 'Index')['Value'].mean() ## Optimized

Index
MSFT     25.633313
SPY     122.300760
TLT      87.219710
Name: Value, dtype: float64

In [19]:
data3['Returns'] = data3['Value'].pct_change()
data3.groupby(level = 'Index')['Returns'].agg([('Average', 'mean'),
                                              ('Volatility', 'std')])

,Average,Volatility
Index,,
MSFT,0.000478,0.018263
SPY,0.001165,0.038465
TLT,0.000027,0.017967


In [20]:
data3.groupby(level = 'Index').agg({'Returns': ['min', 'max', 'std', 'mean'],
                                   'Value': 'mean'}) # Hierarchical columns

Value   Returns                              
             mean       min       max       std      mean
Index                                                    
MSFT    25.633313 -0.117131  0.186047  0.018263  0.000478
SPY    122.300760 -0.098448  1.608635  0.038465  0.001165
TLT     87.219710 -0.678630  0.051662  0.017967  0.000027

### Transformations: transform and apply

More than simple aggregations on each group. For example, let's say we want to get the last 5 observation for each index. 

In [40]:
def top(group):
    return group.iloc[:5]
data3.groupby(level = 'Index').apply(top).sort_index()

Value
Index Index Date                  
MSFT  MSFT  2007-01-03   23.478417
            2007-01-04   23.439102
            2007-01-05   23.305433
            2007-01-08   23.533456
            2007-01-09   23.557044
SPY   SPY   2007-01-03  114.809403
            2007-01-04  115.053042
            2007-01-05  114.135342
            2007-01-08  114.663228
            2007-01-09  114.565777
TLT   TLT   2007-01-03   63.358304
            2007-01-04   63.742467
            2007-01-05   63.465017
            2007-01-08   63.578846
            2007-01-09   63.578846

In [29]:
df = DataFrame({'category': ['a', 'a', 'a', 'a', 'b', 'b', 'b', 'b'], 
                'data': np.random.randn(8),'weights': np.random.rand(8)})
df

,category,data,weights
0,a,-1.090777,0.412978
1,a,0.052670,0.430618
2,a,-1.015201,0.953990
3,a,1.882427,0.015759
4,b,-0.315387,0.128588
5,b,-0.666118,0.936855
6,b,-0.614822,0.712233
7,b,-0.241152,0.388550


Let's compute the annual correlation of the daily returns of the different tickers.

In [30]:
grouped = df.groupby('category').apply(lambda g: np.average(g['data'], weights = g['weights']))
grouped

category
a   -0.753641
b   -0.552208
dtype: float64

In [39]:
def ret_aritmetico(group):
    return group['Value'].pct_change()
data3 = data3.sort_index()
data10 = data3.groupby(level = 'Index', group_keys = False).apply(ret_aritmetico)
data10.groupby(level = 'Index', group_keys= False).apply(top)

Index  Date      
MSFT   2007-01-03         NaN
       2007-01-04   -0.001675
       2007-01-05   -0.005703
       2007-01-08    0.009784
       2007-01-09    0.001002
SPY    2007-01-03         NaN
       2007-01-04    0.002122
       2007-01-05   -0.007976
       2007-01-08    0.004625
       2007-01-09   -0.000850
TLT    2007-01-03         NaN
       2007-01-04    0.006063
       2007-01-05   -0.004353
       2007-01-08    0.001794
       2007-01-09    0.000000
Name: Value, dtype: float64

In [42]:
data10 = data10.dropna()
data10.head()

Index  Date      
MSFT   2007-01-04   -0.001675
       2007-01-05   -0.005703
       2007-01-08    0.009784
       2007-01-09    0.001002
       2007-01-10   -0.010013
Name: Value, dtype: float64

In [51]:
data11 = data10.unstack(0)
data11.head()

Index,MSFT,SPY,TLT
Date,,,
2007-01-04,-0.001675,0.002122,0.006063
2007-01-05,-0.005703,-0.007976,-0.004353
2007-01-08,0.009784,0.004625,0.001794
2007-01-09,0.001002,-0.000850,0.000000
2007-01-10,-0.010013,0.003332,-0.004476


In [52]:
data11.groupby(lambda x: x.year).apply(lambda g: g.corrwith(g['SPY']))

Index,MSFT,SPY,TLT
2007,0.646075,1.0,-0.395089
2008,0.794946,1.0,-0.483310
2009,0.649881,1.0,-0.323391
2010,0.728162,1.0,-0.553058
2011,0.809871,1.0,-0.708705
2012,0.685635,1.0,-0.651584
2013,0.393541,1.0,-0.213911
2014,0.576128,1.0,-0.460401
